In [1]:
# =====================================
# STEP 1. Kutubxonalarni o‘rnatish
# =====================================
!pip install git+https://github.com/openai/whisper.git
!pip install transformers

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-pelcrjg1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-pelcrjg1
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=f6ceae5f40fbb968644b74ef9985393df476b3a42c6eefd7b4cd9c83668dbb20
  Stored in directory: /tmp/pip-ephem-wheel-cache-4f6i7y30/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper


In [7]:
# =====================================
# STEP 2. Audio faylni yuklash
# =====================================
from google.colab import files

print("📂 Iltimos, audio faylni yuklang (masalan: lesson.mp3 yoki lecture.wav)")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print("✅ Yuklangan fayl:", filename)

# =====================================
# STEP 3. Whisper bilan Speech-to-Text
# =====================================
import whisper

print("🔊 Audio fayl transcript qilinmoqda...")
stt_model = whisper.load_model("small")  # "base" tezroq, "small" sifatliroq
result = stt_model.transcribe(filename, fp16=False)
transcript = result["text"]

print("\n--- Transcript (matn) ---\n", transcript[:500], " ...")

# =====================================
# STEP 4. Savol generatsiyasi
# =====================================
from transformers import pipeline

print("\n❓ Savollar generatsiya qilinmoqda...")
qg = pipeline("text2text-generation", model="valhalla/t5-small-qg-hl")

questions = qg(
    transcript,
    max_length=64,
    do_sample=True,
    num_return_sequences=5,
    num_beams=10
)

q_list = [q['generated_text'] for q in questions]

print("\n--- Savollar ---")
for i, q in enumerate(q_list, 1):
    print(f"{i}. {q}")

# =====================================
# STEP 5. Foydalanuvchi savol tanlaydi va javob yozadi
# =====================================
choice = int(input("\n👉 Qaysi savolni tanlaysiz? (raqam kiriting): "))
selected_question = q_list[choice-1]
print("\nTanlangan savol:", selected_question)

student_answer = input("✍️ Javobingizni yozing: ")

# =====================================
# STEP 6. Model javob topadi va feedback beradi
# =====================================
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

model_result = qa_pipeline({
    "context": transcript,
    "question": selected_question
})

model_answer = model_result['answer']

print("\n--- Natija ---")
print("❓ Savol:", selected_question)
print("✅ Model javobi:", model_answer)
print("👨‍🎓 Sizning javobingiz:", student_answer)

# Oddiy feedback (string o‘xshashlik orqali)
from difflib import SequenceMatcher
similarity = SequenceMatcher(None, model_answer.lower(), student_answer.lower()).ratio()

if similarity > 0.7:
    feedback = "👏 To‘g‘ri javob!"
elif similarity > 0.4:
    feedback = "ℹ️ Javob qisman to‘g‘ri."
else:
    feedback = "❌ Javob noto‘g‘ri yoki mos emas."

print("📢 Feedback:", feedback)

📂 Iltimos, audio faylni yuklang (masalan: lesson.mp3 yoki lecture.wav)


Saving english.m4a to english (1).m4a
✅ Yuklangan fayl: english (1).m4a
🔊 Audio fayl transcript qilinmoqda...

--- Transcript (matn) ---
  Do you want to improve your English pronunciation? Well there is a simple trick that can make you sound more English and it is squeezed pronunciation. Want to find out more? Then keep watching! Hello it's Keith once again from the Keith speaking Academy here to help you become a much more confident speaker of English. Now then do you sometimes listen to a native speaker and think what did he say? And then maybe you check the transcript and you go he didn't say that. For example you may hear the  ...

❓ Savollar generatsiya qilinmoqda...


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (1384 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Savollar ---
1. What did Keith do to help you become a more confident speaker of English?
2. What is a squeezed pronunciation trick that can help you become a more confident speaker of English?
3. What did Keith do to help you become a more confident speaker in English?
4. What did Keith do to help you become a more confident speaker?
5. What did Keith do to help you become more confident in English?

👉 Qaysi savolni tanlaysiz? (raqam kiriting): 2

Tanlangan savol: What is a squeezed pronunciation trick that can help you become a more confident speaker of English?
✍️ Javobingizni yozing: speak more fluency


Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



--- Natija ---
❓ Savol: What is a squeezed pronunciation trick that can help you become a more confident speaker of English?
✅ Model javobi: drop the D sound
👨‍🎓 Sizning javobingiz: speak more fluency
📢 Feedback: ❌ Javob noto‘g‘ri yoki mos emas.
